# Data Preparation for Heritage Analysis

## Introduction
Welcome to the first step of your data-driven preservation journey! 

Before we can use advanced algorithms to find patterns or predict deterioration, we must first "clean" and prepare our data. Think of this like preparing a historic surface before applying a conservation treatment: if you don't remove the dirt and loose material first, the treatment won't work.

In this notebook, we will cover three essential steps:
1. **Handling Missing Values**: What do we do when our records are incomplete? (Imputation)
2. **Encoding Categories**: How do we translate words like "Brick" or "Stone" into numbers the computer understands? (One-Hot Encoding)
3. **Standardization**: How do we compare variables with different units, like temperature (degrees) and crack width (millimeters)? (Scaling)

---

## Step 0: Upload Your Data (Google Colab Users)

If you're running this in **Google Colab**, you need to upload the `heritage_data.csv` file first.

**Option 1: Manual Upload**
1. Click the folder icon 📁 on the left sidebar
2. Click the upload button and select `heritage_data.csv`

**Option 2: Use the code below** (Uncomment and run if needed)

### Real-World Example: Why Data Preparation Matters

#### The Scenario

Imagine you have data on 200 historic buildings:
- **Building A**: Year built = 1850, Material = "Brick", Condition = 3.5
- **Building B**: Year built = 1920, Material = "Stone", Condition = ??? (missing!)
- **Building C**: Year built = 1875, Material = "brick" (lowercase!), Condition = 4.2

#### Problems You'll Encounter

**1. Missing Data**
- Building B has no condition rating
- **Solution**: Fill with average, or remove that row

**2. Inconsistent Formatting**
- "Brick" vs. "brick" — computer sees these as different!
- **Solution**: Standardize to lowercase

**3. Different Scales**
- Year built: 1800-2000 (range = 200)
- Condition: 1-5 (range = 4)
- **Problem**: Models think "year" is 50x more important just because numbers are bigger!
- **Solution**: Standardize (make all variables comparable)

**4. Categorical Variables**
- Material = "Brick", "Stone", "Wood"
- **Problem**: Models need numbers, not words
- **Solution**: One-hot encoding (create dummy variables)

#### What This Notebook Does

We'll walk through each problem with **concrete examples** and **ready-to-use code**:

| Step | What It Fixes | Example |
|------|---------------|----------|
| **1. Load Data** | Get your spreadsheet into Python | CSV → DataFrame |
| **2. Check Missing** | Find gaps in your data | "15 buildings missing moisture data" |
| **3. Handle Missing** | Fill or remove gaps | Fill with average moisture |
| **4. Encode Categories** | Convert text to numbers | "Brick" → Material_Brick=1 |
| **5. Standardize** | Make scales comparable | Year 1850 → -1.2, Year 2000 → +1.8 |
| **6. Save Clean Data** | Export for next steps | processed_data.csv |

#### Before vs. After

**Before (raw data)**:
```
Building_ID | Year | Material | Moisture | Condition
001         | 1850 | Brick    | 45.2     | 3.5
002         | 1920 | Stone    | ???      | 4.1
003         | 1875 | brick    | 67.8     | 2.9
```

**After (clean data)**:
```
Building_ID | Year_Std | Material_Brick | Material_Stone | Moisture_Std | Condition
001         | -1.2     | 1              | 0              | -0.5         | 3.5
002         | 0.3      | 0              | 1              | 0.0 (filled) | 4.1
003         | -0.8     | 1              | 0              | 1.2          | 2.9
```

Now the data is ready for analysis!

Let's get started:

In [1]:
# Uncomment the lines below if you want to upload via code
# from google.colab import files
# uploaded = files.upload()
# print("File uploaded successfully!")

In [2]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Load the dataset with error handling
try:
    df = pd.read_csv('heritage_data.csv')
    print(f"✅ Dataset loaded successfully!")
    print(f"Dataset shape: {df.shape} (Rows, Columns)")
except FileNotFoundError:
    print("❌ ERROR: 'heritage_data.csv' not found.")
    print("Please upload the file using the instructions above.")
    raise

✅ Dataset loaded successfully!
Dataset shape: (200, 17) (Rows, Columns)


## Exploring the Data

Before we clean anything, let's understand what we have.

In [3]:
# Show the first few rows
print("First 5 rows of the dataset:")
display(df.head())

First 5 rows of the dataset:


,Building_ID,District_ID,Construction_Year,Material_Type,Foundation_Type,Avg_Temp_C,Temp_Range_C,Annual_Rainfall_mm,Humidity_Percent,Freeze_Thaw_Cycles,Soil_Moisture_Index,Crack_Width_mm,Salt_Deposition_g_m2,Condition_Rating,Intervention_Urgency,Latitude,Longitude
0,B001,West,1897,Stone,Pile,22.615620,11.779304,1076.431798,70.989601,9,13.517939,4.840544,5.081838,3,100.0,40.849671,-77.764221
1,B002,Central,1938,Brick,Deep,17.694550,9.071361,929.741978,58.789251,13,16.871188,4.966796,2.995625,4,100.0,40.786174,-77.743922
2,B003,East,1943,Brick,Shallow,9.813769,10.978399,766.576384,61.179528,5,12.713060,3.149923,6.481649,3,100.0,40.864769,-77.691695
3,B004,Central,1896,Concrete,Shallow,14.048307,6.246659,829.342737,30.026990,11,11.115060,3.221341,3.973573,3,100.0,40.952303,-77.694620
4,B005,Central,1923,Brick,Pile,10.621909,12.772081,1041.301793,73.744915,6,18.451470,5.083610,4.542800,4,100.0,40.776585,-77.937767


### Data Dictionary

Here's what each column means:

| Column Name | Description | Units/Type |
|------------|-------------|------------|
| `Building_ID` | Unique identifier for each building | Text (e.g., B001) |
| `District_ID` | Geographic district | Category (North, South, East, West, Central) |
| `Construction_Year` | Year the building was constructed | Year |
| `Material_Type` | Primary construction material | Category (Brick, Stone, Concrete, Wood) |
| `Foundation_Type` | Type of foundation | Category (Shallow, Deep, Pile) |
| `Avg_Temp_C` | Average annual temperature | Degrees Celsius |
| `Temp_Range_C` | Seasonal temperature variation | Degrees Celsius |
| `Annual_Rainfall_mm` | Total annual rainfall | Millimeters |
| `Humidity_Percent` | Average relative humidity | Percentage (0-100) |
| `Freeze_Thaw_Cycles` | Number of freeze-thaw events per year | Count |
| `Soil_Moisture_Index` | Composite measure of ground moisture | Index (0-20) |
| `Crack_Width_mm` | Maximum observed crack width | Millimeters |
| `Salt_Deposition_g_m2` | Salt accumulation on surfaces | Grams per square meter |
| `Condition_Rating` | Overall condition assessment | Scale 1-5 (1=Good, 5=Poor) |
| `Intervention_Urgency` | Priority score for intervention | Scale 0-100 |

In [4]:
# Summary statistics for numerical columns
print("Summary Statistics:")
display(df.describe())

# Data types and non-null counts
print("\nData Types and Missing Values:")
print(df.info())

Summary Statistics:


,Construction_Year,Avg_Temp_C,Temp_Range_C,Annual_Rainfall_mm,Humidity_Percent,Freeze_Thaw_Cycles,Soil_Moisture_Index,Crack_Width_mm,Salt_Deposition_g_m2,Condition_Rating,Intervention_Urgency,Latitude,Longitude
count,200.00000,200.000000,200.000000,200.000000,200.000000,200.000000,188.000000,192.000000,190.000000,200.000000,200.000000,200.000000,200.000000
mean,1879.89500,14.733929,10.299198,821.207687,61.072402,9.895000,14.236467,4.609289,5.060827,3.285000,99.801352,40.795923,-77.791413
std,44.41438,5.030212,2.874835,191.102818,15.925291,3.270698,3.234829,0.880777,2.022916,0.798098,1.790347,0.093100,0.098700
min,1800.00000,1.515567,3.754212,230.291476,20.000000,2.000000,6.367272,1.831631,0.124667,1.000000,77.117183,40.538025,-78.124127
25%,1845.00000,11.372754,8.306390,702.729296,49.374589,8.000000,12.029914,4.045472,3.729589,3.000000,100.000000,40.729487,-77.860584
50%,1889.00000,14.695839,10.315523,827.968336,61.571825,10.000000,14.149276,4.611791,5.038764,3.000000,100.000000,40.799581,-77.792116
75%,1917.25000,18.100736,12.044795,941.703655,71.088183,12.000000,16.789422,5.242084,6.342448,4.000000,100.000000,40.850085,-77.731277
max,1949.00000,27.866799,17.897146,1232.650945,98.695640,19.000000,20.000000,6.991440,11.219837,5.000000,100.000000,41.072017,-77.414727



Data Types and Missing Values:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Building_ID           200 non-null    object 
 1   District_ID           200 non-null    object 
 2   Construction_Year     200 non-null    int64  
 3   Material_Type         200 non-null    object 
 4   Foundation_Type       200 non-null    object 
 5   Avg_Temp_C            200 non-null    float64
 6   Temp_Range_C          200 non-null    float64
 7   Annual_Rainfall_mm    200 non-null    float64
 8   Humidity_Percent      200 non-null    float64
 9   Freeze_Thaw_Cycles    200 non-null    int64  
 10  Soil_Moisture_Index   188 non-null    float64
 11  Crack_Width_mm        192 non-null    float64
 12  Salt_Deposition_g_m2  190 non-null    float64
 13  Condition_Rating      200 non-null    int64  
 14  Intervention_Urgency  200 non-null    floa

## 1. Handling Missing Values

### The Problem
Real-world heritage data is rarely perfect. You might have missing sensor readings due to a power outage, or a condition survey where some fields were left blank. Machine learning algorithms generally cannot handle blank spaces—they need a number for every input.

### The Solution: Imputation
Instead of throwing away valuable data just because one number is missing, we can make an educated guess to fill in the gap. This is called **imputation**.
- For **numerical** data (like temperature), we often fill gaps with the **average (mean)** of that column.
- For **categorical** data (like material type), we often fill gaps with the **most frequent** value (the mode).

Think of this like retouching a small loss in a painting: you use the surrounding colors to infer what should be there, rather than leaving a white hole.

In [5]:
# Check for missing values
print("Missing values per column:")
missing_counts = df.isnull().sum()
print(missing_counts[missing_counts > 0])

if missing_counts.sum() == 0:
    print("\n✅ No missing values found!")
else:
    print(f"\n⚠️ Total missing values: {missing_counts.sum()}")

Missing values per column:
Soil_Moisture_Index     12
Crack_Width_mm           8
Salt_Deposition_g_m2    10
dtype: int64

⚠️ Total missing values: 30


## 2. Encoding Categories

### The Problem
Computers are excellent at math, but they don't understand words. If we feed the word "Brick" into a mathematical equation, it will fail. We need to convert these text labels into numbers.

### The Solution: One-Hot Encoding
We could just assign numbers (Brick=1, Stone=2, Wood=3), but that implies an order (is Wood "greater" than Brick?). Instead, we use **One-Hot Encoding**.

This creates a new column for each category. 
- Is_Brick: 1 if yes, 0 if no
- Is_Stone: 1 if yes, 0 if no

It's like a switchboard where only the relevant light is turned on.

## 3. Standardization (Scaling)

### The Problem
Imagine comparing "Annual Rainfall" (e.g., 800 mm) with "Crack Width" (e.g., 0.5 mm). The rainfall number is huge compared to the crack width! If we don't fix this, the algorithm might think rainfall is 1600 times more important just because the number is bigger.

### The Solution: Scaling
We **standardize** the data so that every variable is on the same playing field. We adjust the values so that the average is 0 and the spread (standard deviation) is 1. 
- A value of +1 means "somewhat higher than average".
- A value of -2 means "much lower than average".

This ensures that a 10% change in humidity is treated with the same importance as a 10% change in crack width.

In [6]:
# Let's see how this works manually on one column first
example_col = df['Avg_Temp_C'].head()
print("Original Values:")
print(example_col)

# Manual Calculation: (Value - Mean) / Standard Deviation
scaled_col = (example_col - example_col.mean()) / example_col.std()
print("\nScaled Values (Manual):")
print(scaled_col)

print("\n💡 Notice: The values are now small numbers centered around 0.")

Original Values:
0    22.615620
1    17.694550
2     9.813769
3    14.048307
4    10.621909
Name: Avg_Temp_C, dtype: float64

Scaled Values (Manual):
0    1.445577
1    0.516495
2   -0.971371
3   -0.171904
4   -0.818797
Name: Avg_Temp_C, dtype: float64

💡 Notice: The values are now small numbers centered around 0.


## Putting it all together: The Pipeline

Doing that math manually for every column is tedious. In Python, we use a **Pipeline** to automate it. This ensures that every piece of data goes through the exact same cleaning process.

In [7]:
# Define which columns are which
numeric_features = ['Construction_Year', 'Avg_Temp_C', 'Temp_Range_C', 
                    'Annual_Rainfall_mm', 'Humidity_Percent', 'Freeze_Thaw_Cycles', 
                    'Soil_Moisture_Index', 'Crack_Width_mm', 'Salt_Deposition_g_m2']

categorical_features = ['District_ID', 'Material_Type', 'Foundation_Type']

# Create transformers (the "workers" that will process the data)

# For numbers: Fill missing with Mean -> Scale to standard range
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

# For categories: Fill missing with Most Frequent -> Convert to One-Hot numbers
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Combine them into one master processor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Apply the transformation
print("Processing data...")
X_processed = preprocessor.fit_transform(df)

# Get the new column names (especially for the one-hot encoded categories)
ohe = preprocessor.named_transformers_['cat']['encoder']
cat_feature_names = ohe.get_feature_names_out(categorical_features)
feature_names = numeric_features + list(cat_feature_names)

# Create a nice clean DataFrame with the results
df_processed = pd.DataFrame(X_processed, columns=feature_names)

# Add back the ID and Outcome variables (we don't scale outcomes usually)
df_processed['Building_ID'] = df['Building_ID'].values
df_processed['Condition_Rating'] = df['Condition_Rating'].values
df_processed['Intervention_Urgency'] = df['Intervention_Urgency'].values

# Also pass through coordinates if present
if 'Latitude' in df.columns:
    df_processed['Latitude'] = df['Latitude'].values
if 'Longitude' in df.columns:
    df_processed['Longitude'] = df['Longitude'].values

print("✅ Done! Processed Data Shape:", df_processed.shape)
display(df_processed.head())

Processing data...
✅ Done! Processed Data Shape: (200, 26)


,Construction_Year,Avg_Temp_C,Temp_Range_C,Annual_Rainfall_mm,Humidity_Percent,Freeze_Thaw_Cycles,Soil_Moisture_Index,Crack_Width_mm,Salt_Deposition_g_m2,District_ID_Central,...,Material_Type_Stone,Material_Type_Wood,Foundation_Type_Deep,Foundation_Type_Pile,Foundation_Type_Shallow,Building_ID,Condition_Rating,Intervention_Urgency,Latitude,Longitude
0,0.386089,1.570802,0.516141,1.338884,0.624295,-0.274329,-0.229714,0.268672,0.010684,0.0,...,1.0,0.0,0.0,1.0,0.0,B001,3,100.0,40.849671,-77.764221
1,1.311530,0.590045,-0.428170,0.569362,-0.143726,0.951721,0.842321,0.415353,-1.050193,1.0,...,0.0,0.0,1.0,0.0,0.0,B002,4,100.0,40.786174,-77.743922
2,1.424389,-0.980576,0.236850,-0.286591,0.006744,-1.500378,-0.487034,-1.695495,0.722513,0.0,...,0.0,0.0,0.0,0.0,1.0,B003,3,100.0,40.864769,-77.691695
3,0.363518,-0.136643,-1.413197,0.042676,-1.954333,0.338696,-0.997915,-1.612522,-0.552889,1.0,...,0.0,0.0,0.0,0.0,1.0,B004,3,100.0,40.952303,-77.694620
4,0.972954,-0.819516,0.862341,1.154595,0.797745,-1.193866,1.347537,0.551068,-0.263426,1.0,...,0.0,0.0,0.0,1.0,0.0,B005,4,100.0,40.776585,-77.937767


## Saving the Result
Now that our data is clean, filled, and scaled, we save it to a new file. We will use this `processed_data.csv` for all our analysis in the next notebooks.

In [8]:
df_processed.to_csv('processed_data.csv', index=False)
print("✅ Saved processed_data.csv")
print(f"   - Original columns: {len(df.columns)}")
print(f"   - Processed columns: {len(df_processed.columns)}")
print(f"   - Rows: {len(df_processed)}")

✅ Saved processed_data.csv
   - Original columns: 17
   - Processed columns: 26
   - Rows: 200


## Real-World Challenge: Imbalanced Data

### The Scenario
Our synthetic dataset is "perfectly balanced" (roughly equal numbers of buildings in each condition). In the real world, heritage data is often **imbalanced**:
- **95%** of your buildings might be in "Good" condition.
- Only **5%** might be in "Poor" condition.

### The Problem
If you train a model on this, it gets lazy! It learns to just guess "Good" for every single building.
- **Result**: It achieves **95% accuracy** (wow!).
- **Reality**: It misses **100% of the problems** (useless!).

### What To Do (If You Have This Problem)

We won't cover the code here to keep things simple, but here are the keywords to search for when you encounter this:

1. **Resampling**: Changing your dataset to be more balanced.
   - **Oversampling**: Duplicating the rare examples (or creating synthetic ones using a tool called **SMOTE**).
   - **Undersampling**: Using fewer of the abundant examples.

2. **Better Metrics**: Never trust "Accuracy" alone for imbalanced data.
   - Use **Precision** and **Recall** (we'll discuss these in Notebook 3).
   - A model with 80% accuracy that finds half the defects is better than a 95% accurate model that finds none!

**Key Takeaway**: If your data is highly imbalanced, don't just run the standard analysis. Look up "handling imbalanced data with SMOTE".

## Troubleshooting

**Common Issues:**

1. **FileNotFoundError**: Make sure `heritage_data.csv` is uploaded to your workspace.
2. **KeyError (column not found)**: Check that your CSV has the expected column names.
3. **Memory Error**: If your dataset is very large (>10,000 rows), you may need more RAM. Try using a smaller sample.

**Need Help?** Check that:
- Your CSV file is properly formatted
- Column names match exactly (case-sensitive)
- There are no special characters in the data

## Next Steps

Now that your data is prepared, you can proceed to:

1. **Notebook 2: Factor Analysis** - Identify hidden patterns and group related variables
2. **Notebook 3: Feature Importance** - Determine which variables drive deterioration
3. **Notebook 4: Visualization** - Create publication-ready charts

Remember to use the `processed_data.csv` file you just created!